In [2]:
import os
import random
from PIL import Image, ImageDraw
from math import sin, cos, tan, asin, acos, atan, sinh, cosh, tanh, pi
import sympy as sp
import random

CANVAS_WIDTH = 500
CANVAS_HEIGHT = 500
NUM_IMAGES = 1000
COMPONENTS_FOLDER = "./components"
OUTPUT_FOLDER = "./generated_dataset"
FONT_COLOR = (0, 0, 0)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

USE_DECIMAL = 30  # 30% chance to use decimals
USE_PI = 40  # 40% chance to include π
TRIGONOMETRIC_TYPES = ["sin", "cos", "tan", "sec", "cosec", "cot", "asin", "acos", "atan", "sinh", "cosh", "tanh"]
TRIGONOMETRIC_TYPES = ["sin", "cos", "tan"]
EXPRESSION_TYPE_WEIGHTS = [40, 20, 30, 10]  # Weighted probabilities for ["complete", "trigonometric", "calculus", "number"]
NUMBER_WEIGHTS = [70, 20, 10]  # Higher probability for smaller numbers
POWER_WEIGHTS = [50, 30, 10, 5, 5]  # Higher probability for powers 1, 2
COMPLEXITY_WEIGHTS = [60, 30, 10]  # Higher probability for simpler expressions

def sec(x):
    return 1 / cos(x)
def cosec(x):
    return 1 / sin(x)
def cot(x):
    return 1 / tan(x)

def load_random_image(folder):
    files = os.listdir(folder)
    if not files:
        raise ValueError(f"No files found in folder {folder}")
    random_file = random.choice(files)
    return Image.open(os.path.join(folder, random_file))


def weighted_random(weights, choices):
    return random.choices(choices, weights=weights, k=1)[0]

def generate_number():
    num_terms = random.choices([2, 3, 4], weights=[60, 30, 10], k=1)[0]
    numbers = [
        random.uniform(1, 50) if random.randint(1, 100) <= USE_DECIMAL else random.randint(1, 50)
        for _ in range(num_terms)
    ]
    lhs = " + ".join([str(round(num, 1)) for num in numbers])
    rhs = round(sum(numbers), 1)
    return [lhs, "=", str(rhs)]

def generate_trigonometric_expression():
    trig_function = random.choice(TRIGONOMETRIC_TYPES)
    coeff = weighted_random(None, range(1,11))
    include_pi = random.randint(1, 100) <= USE_PI

    lhs = [f"{trig_function}", f"({coeff}"]
    if include_pi:
        lhs.append("*pi")

    lhs.append(")")
    rhs = str(round(eval("".join(lhs)), 2))

    return lhs + ["="] + [rhs]

def solve_diff_expression(expr_list):
    expr_string = ''.join(expr_list)
    var_str = expr_list[0].split('/')[1][1:]

    var = sp.symbols(var_str)
    
    expr_string = expr_string.replace(f'd/d{var_str}', '')
    expr_string = expr_string.replace(f'^', '**')
    expr_string = expr_string.replace(f'{var_str}', f'*{var_str}')

    result = sp.diff(expr_string, var)
    result = str(result).replace('**', '^')
    result = result.replace(f'*{var_str}', f'{var_str}')
    return result

def solve_int_expression(expr_list):
    var_str = expr_list[-1][1]
    expr_list = expr_list[1:-1]
    expr_string = ''.join(expr_list)

    var = sp.symbols(var_str)
    
    expr_string = expr_string.replace(f'^', '**')
    expr_string = expr_string.replace(f'{var_str}', f'*{var_str}')

    result = sp.integrate(expr_string, var)
    result = str(result).replace('**', '^')
    result = result.replace(f'*{var_str}', f'{var_str}')
    return result

def generate_calculus_expression():
    calculus_type = random.choice(["d/d", "integral"])
    selected_var = random.choice(["x", "y", "z"])

    terms = [f"{weighted_random(None, range(1,11))}{selected_var}^{weighted_random(POWER_WEIGHTS, range(1, 6))}" for _ in range(weighted_random(COMPLEXITY_WEIGHTS, range(1, 4)))]
    new_terms = []
    for i, item in enumerate(terms):
        if not i == 0:
            new_terms.append("+")
        new_terms.append(item)

    if calculus_type == "d/d":
        lhs = [f"d/d{selected_var}", "("] + new_terms + [")"]
        rhs = [solve_diff_expression(lhs)]
    else:
        lhs = ["∫", "("] + new_terms + [")", f"d{selected_var}"]
        rhs = [solve_int_expression(lhs)]

    return lhs + ["="] + rhs

def generate_complete_equation():
    variables = []
    var1, var2 = random.sample(["x", "y", "z"], 2)
    coeff1, coeff2 = random.randint(1, 10), random.randint(1, 10)

    var1_value = random.randint(1, 50)
    variables+=[f'{var1}', "=", f"{var1_value}", "\n"]

    var2_value = random.randint(1, 50)
    variables+=[f'{var2}', "=", f"{var2_value}", "\n"]

    lhs = [f"{coeff1}{var1}", "+", f"{coeff2}{var2}"]
    rhs = [str(coeff1 * var1_value + coeff2 * var2_value)]

    return variables + lhs + ["="] + rhs


def generate_incomplete_expression():
    expression_type = weighted_random(EXPRESSION_TYPE_WEIGHTS, ["complete", "trigonometric", "calculus", "number"])
    if expression_type == "complete":
        expression = generate_complete_equation({})
    elif expression_type == "trigonometric":
        expression = generate_trigonometric_expression({})
    elif expression_type == "calculus":
        expression = generate_calculus_expression({})
    else:
        expression = generate_number()
    return expression[:-1] + ["="]


def generate_training_data():
    variables = {}
    for i in range(NUM_IMAGES):
        canvas = Image.new("RGB", (CANVAS_WIDTH, CANVAS_HEIGHT), "white")

        num_complete = random.randint(0, 7)
        for _ in range(num_complete):
            expression_type = weighted_random(EXPRESSION_TYPE_WEIGHTS, ["complete", "trigonometric", "calculus", "number"])
            if expression_type == "complete":
                expression = generate_complete_equation(variables)
            elif expression_type == "trigonometric":
                expression = generate_trigonometric_expression(variables)
            elif expression_type == "calculus":
                expression = generate_calculus_expression(variables)
            else:
                expression = generate_number()

            print("Expression:", expression)

        incomplete_expression = generate_incomplete_expression()
        print("Incomplete:", incomplete_expression)

# if __name__ == "__main__":
#     generate_training_data()

In [8]:
def generate_calculus_expression():
    calculus_type = random.choice(["d/d", "integral"])
    selected_var = random.choice(["x", "y", "z"])

    terms = [f"{weighted_random(None, range(1,11))}{selected_var}^{weighted_random(POWER_WEIGHTS, range(1, 6))}" for _ in range(weighted_random(COMPLEXITY_WEIGHTS, range(1, 4)))]
    new_terms = []
    for i, item in enumerate(terms):
        if not i == 0:
            new_terms.append("+")
        new_terms.append(item)

    if calculus_type == "d/d":
        lhs = [f"d/d{selected_var}", "("] + new_terms + [")"]
        rhs = [solve_diff_expression(lhs)]
    else:
        lhs = ["∫", "("] + new_terms + [")", f"d{selected_var}"]
        rhs = [solve_int_expression(lhs)]

    return lhs + ["="] + rhs

generate_calculus_expression()

['∫', '(', '4z^3', ')', 'dz', '=', 'z^4']

In [ ]:
import tkinter as tk
from tkinter import simpledialog
import time
import json

class SimulatedDrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simulated Writing Canvas")

        # Initialize variables
        self.pen_color = "black"
        self.actions = []  # Log of actions
        self.current_action = None

        # Canvas
        self.canvas = tk.Canvas(root, bg="white", width=800, height=600)
        self.canvas.pack(fill=tk.BOTH, expand=True)

        # Buttons and tools
        button_frame = tk.Frame(root)
        button_frame.pack()

        text_button = tk.Button(button_frame, text="Write Text", command=self.add_text)
        text_button.pack(side=tk.LEFT, padx=5)

        simulate_button = tk.Button(button_frame, text="Simulate", command=self.simulate_writing)
        simulate_button.pack(side=tk.LEFT, padx=5)

        save_button = tk.Button(button_frame, text="Save Actions", command=self.save_actions)
        save_button.pack(side=tk.LEFT, padx=5)

        load_button = tk.Button(button_frame, text="Load Actions", command=self.load_actions)
        load_button.pack(side=tk.LEFT, padx=5)

        self.speed_factor = 1  # Default speed factor

    def add_text(self):
        """Prompt for text and add it to the canvas."""
        text = simpledialog.askstring("Input Text", "Enter the text to write:")
        if text:
            x, y = simpledialog.askinteger("X Position", "Enter the X coordinate:"), simpledialog.askinteger(
                "Y Position", "Enter the Y coordinate:"
            )
            start_time = time.time()
            self.canvas.create_text(x, y, text=text, fill=self.pen_color, font=("Arial", 20))
            elapsed_time = time.time() - start_time

            # Log action
            self.actions.append({
                "type": "text",
                "x": x,
                "y": y,
                "text": text,
                "color": self.pen_color,
                "time": elapsed_time,
            })

    def simulate_writing(self):
        """Simulate the writing by replaying actions."""
        for action in self.actions:
            if action["type"] == "text":
                self.root.update()
                time.sleep(action["time"] / self.speed_factor)  # Simulate the original time or adjust by speed factor
                self.canvas.create_text(action["x"], action["y"], text=action["text"], fill=action["color"], font=("Arial", 20))

    def save_actions(self):
        """Save actions to a file."""
        with open("actions.json", "w") as file:
            json.dump(self.actions, file)
        print("Actions saved to actions.json")

    def load_actions(self):
        """Load actions from a file."""
        try:
            with open("actions.json", "r") as file:
                self.actions = json.load(file)
            print("Actions loaded from actions.json")
        except FileNotFoundError:
            print("No saved actions found!")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = SimulatedDrawingApp(root)
    root.mainloop()


In [47]:
import tkinter as tk
from tkinter import colorchooser
from PIL import Image, ImageDraw

class DrawingApp:
    def __init__(self, root, save_folder):
        self.root = root
        self.root.title("Drawing Canvas")

        # Save folder for images
        self.save_folder = save_folder
        self.image_counter = 1

        # Initialize variables
        self.pen_color = "black"
        self.eraser_on = False
        self.old_x = None
        self.old_y = None

        # Create a blank image for drawing
        self.image = Image.new("RGB", (800, 600), "white")
        self.draw = ImageDraw.Draw(self.image)

        # Canvas
        self.canvas = tk.Canvas(root, bg="white", width=800, height=600)
        self.canvas.pack(fill=tk.BOTH, expand=True)

        # Buttons and tools
        button_frame = tk.Frame(root)
        button_frame.pack()

        color_button = tk.Button(button_frame, text="Select Color", command=self.choose_color)
        color_button.pack(side=tk.LEFT, padx=5)

        eraser_button = tk.Button(button_frame, text="Eraser", command=self.toggle_eraser)
        eraser_button.pack(side=tk.LEFT, padx=5)

        reset_button = tk.Button(button_frame, text="Reset", command=self.reset_canvas)
        reset_button.pack(side=tk.LEFT, padx=5)

        # Bind mouse events
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset_coordinates)

        # Save the canvas when the app is closed
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def choose_color(self):
        """Choose pen color using color chooser dialog."""
        color = colorchooser.askcolor()[1]
        if color:
            self.pen_color = color
            self.eraser_on = False  # Turn off eraser when color is selected

    def toggle_eraser(self):
        """Toggle eraser mode."""
        self.eraser_on = not self.eraser_on
        if self.eraser_on:
            self.pen_color = "white"  # Set pen color to match canvas background

    def reset_canvas(self):
        """Clear the canvas and save the current image."""
        self.save_image()
        self.canvas.delete("all")
        self.image = Image.new("RGB", (800, 600), "white")
        self.draw = ImageDraw.Draw(self.image)

    def paint(self, event):
        """Draw on the canvas and the underlying image."""
        if self.old_x and self.old_y:
            line_width = 5 if not self.eraser_on else 20
            # Draw on canvas
            self.canvas.create_line(
                self.old_x, self.old_y, event.x, event.y,
                width=line_width, fill=self.pen_color, capstyle=tk.ROUND, smooth=True
            )
            # Draw on the image
            self.draw.line(
                [(self.old_x, self.old_y), (event.x, event.y)],
                fill=self.pen_color, width=line_width
            )
        self.old_x = event.x
        self.old_y = event.y

    def reset_coordinates(self, event):
        """Reset the mouse position coordinates."""
        self.old_x = None
        self.old_y = None

    def save_image(self):
        """Save the current image to the specified folder."""
        file_path = f"{self.save_folder}/drawing_{self.image_counter}.png"
        self.image.save(file_path)
        print(f"Image saved: {file_path}")
        self.image_counter += 1

    def on_close(self):
        """Save the current canvas before closing the app."""
        self.save_image()
        self.root.destroy()

# Run the application
if __name__ == "__main__":
    save_folder = "./drawings"  # Specify your desired folder for saving images
    import os
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    root = tk.Tk()
    app = DrawingApp(root, save_folder)
    root.mainloop()


Image saved: ./drawings/drawing_1.png
Image saved: ./drawings/drawing_2.png


In [46]:
import tkinter as tk

class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drawing Canvas")
        self.root.geometry("800x600")
        self.root.config(bg="black")

        # Initialize variables
        self.pen_color = "white"
        self.eraser_on = False
        self.eraser_size = 10

        # Frame for buttons
        button_frame = tk.Frame(root, bg="black")
        button_frame.pack(pady=10)

        drop_down_frame = tk.Frame(root, bg="black")
        drop_down_frame.pack(pady=10)
        
        # Canvas
        self.canvas = tk.Canvas(root, bg="black", width=800, height=600)
        self.canvas.pack(fill=tk.BOTH, expand=True)

        self.color_button = tk.Button(
            button_frame, text="Color", width=10, height=2, command=self.toggle_color_palette
        )
        self.color_button.pack(side=tk.LEFT, padx=10)

        # Color buttons
        # Color palette frame (hidden initially)
        self.color_palette_frame = tk.Frame(button_frame, bg="black")
        self.color_palette_frame.pack(pady=5, fill=tk.X)
        self.color_palette_frame.pack_forget()
        colors = ["white", "black", "red", "green", "blue", "yellow", "purple", "orange"]
        self.color_buttons = []
        for color in colors:
            color_button = tk.Button(
                self.color_palette_frame, bg=color, width=4, height=2,
                command=lambda c=color: self.choose_color(c)
            )
            color_button.pack(side=tk.LEFT, padx=5)
            self.color_buttons.append(color_button)

        # Eraser button
        eraser_button = tk.Button(
            button_frame, text="Eraser", width=10, height=2,
            command=self.toggle_eraser
        )
        eraser_button.pack(side=tk.LEFT, padx=5)

        # Eraser size buttons
        self.eraser_size_frame = tk.Frame(button_frame, bg="black")
        self.eraser_size_frame.pack(pady=5, fill=tk.X)
        self.eraser_size_frame.pack_forget()

        eraser_sizes = [5, 10, 15, 20]
        self.eraser_size_buttons = []
        for size in eraser_sizes:
            size_button = tk.Button(
                self.eraser_size_frame, text=str(size), width=4, height=2,
                command=lambda s=size: self.set_eraser_size(s)
            )
            size_button.pack(side=tk.LEFT, padx=5)
            self.eraser_size_buttons.append(size_button)

        # Reset button
        reset_button = tk.Button(button_frame, text="Reset", width=10, height=2, command=self.reset_canvas)
        reset_button.pack(side=tk.LEFT, padx=5)

        # Bindings
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset_coordinates)

        self.old_x = None
        self.old_y = None

    def toggle_color_palette(self):
        if self.color_palette_frame.winfo_ismapped():
            self.color_palette_frame.pack_forget()
        else:
            self.color_palette_frame.pack(pady=5, fill=tk.X)

    def choose_color(self, color):
        self.pen_color = color
        self.eraser_on = False
        self.color_palette_frame.pack_forget()

    def toggle_eraser(self):
        self.eraser_on = not self.eraser_on
        if self.eraser_on:
            # if self.eraser_size_frame.winfo_ismapped():
            #     self.eraser_size_frame.pack_forget()
            # else:
            #     self.eraser_size_frame.pack(pady=5, fill=tk.X)
            if not self.eraser_size_frame.winfo_ismapped():
                self.eraser_size_frame.pack(pady=5, fill=tk.X)
            self.pen_color = "black"  # Set pen color to match canvas background
            for btn in self.eraser_size_buttons:
                btn.pack(side=tk.LEFT, padx=5)  # Show size buttons
        else:
            for btn in self.eraser_size_buttons:
                btn.pack_forget()  # Hide size buttons
            self.eraser_size_frame.pack_forget()

    def set_eraser_size(self, size):
        self.eraser_size = size
        self.eraser_size_frame.pack_forget()

    def reset_canvas(self):
        self.canvas.delete("all")

    def paint(self, event):
        if self.old_x and self.old_y:
            line_width = self.eraser_size if self.eraser_on else 5
            self.canvas.create_line(
                self.old_x, self.old_y, event.x, event.y,
                width=line_width, fill=self.pen_color, capstyle=tk.ROUND, smooth=True
            )
        self.old_x = event.x
        self.old_y = event.y

    def reset_coordinates(self, event):
        self.old_x = None
        self.old_y = None

root = tk.Tk()
app = DrawingApp(root)
root.mainloop()